### Import packages

In [191]:
#import the packages
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import requests

### Define constants

In [192]:
#define constants
base_url = 'https://api.foursquare.com/v2'
client_id = '4YYUY0FWACWQSMRZS1SWOQKU4J5T23CFDEHFDJUZGAJBJQL2'
client_secret = 'J2MKJZVR4HMIPD2YXI1VPVNQAQN4012GJTJKGQOJ31YBK240'
wikiUrl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### Fetch Wikipedia article

In [193]:
#Fetch Wikipedia article
resp = requests.get(wikiUrl)
html_text = resp.text

### Parse and build data frame

In [194]:
#Parse and build data frame
soup = BeautifulSoup(html_text, 'html.parser')
postal_codeTable = soup.find('table')
rows = postal_codeTable.find_all('tr')
header = []
for th in rows[0].find_all('th'):
    header.append(th.text.replace('\n',''))
df_postalcode = pd.DataFrame(columns = header)
rows = rows[1:]
#print(rows)
rownum  = 0
for row in rows:
    df_row = []
    for td in row.find_all('td'):
        df_row.append(td.text.replace('\n',''))
    #print(df_row)
    df_postalcode.loc[rownum] = df_row
    rownum = rownum + 1
df_postalcode.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data clean up per instruction -
+  Remove those where borough is 'Not assigned'
+  Group the neighbourhoods
+  Replace Neighbourhood with Borough when Neighbourhood is not assigned

In [195]:
#filter out those where borough is 'Not assigned'
df_postalcode = df_postalcode[df_postalcode['Borough'] != 'Not assigned'].reset_index()
df_postalcode.drop('index', axis=1,inplace=True)

#Group the neighbourhoods
df_postalcode = pd.DataFrame(df_postalcode.groupby(['Postcode','Borough'],axis=0)
                             .apply(lambda x: ','.join(x['Neighbourhood'].values)).reset_index())

df_postalcode.columns = ['Postcode','Borough','Neighbourhood']

#Replace Neighbourhood with Borough when Neighbourhood is not assigned
df_postalcode['Neighbourhood'].where(cond=df_postalcode['Neighbourhood'] != 'Not assigned', other=df_postalcode['Borough'] 
                    ,inplace = True
                    ,axis=0)


### Shape of the final DataFrame

In [196]:
df_postalcode.shape

(103, 3)